In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
import torch
import torch.nn as nn
import torch.nn.functional as F
from mamba_test0729 import Mamba, MambaConfig
import argparse
from datetime import datetime
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr
import random
from sklearn.preprocessing import StandardScaler
from scipy.stats import spearmanr


/home/luchinwei/anaconda3/envs/mambastock/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
save_dir = '0730_2330/'
use_cuda = True
seed_num = random.randint(0, 999999)
n_steps=10
window=15
patience = 100
epochs = 1000
# loss_fnc = F.smooth_l1_loss
# loss_fnc = None

lr = 0.0005
wd = 1e-5
hidden = 16
layer = 2
n_test = 350
ts_code = 2330
risk_free = 0.017

In [ ]:
def evaluation_metric(y_test,y_hat):
    MSE = mean_squared_error(y_test, y_hat)
    RMSE = MSE**0.5
    MAE = mean_absolute_error(y_test,y_hat)
    R2 = r2_score(y_test,y_hat)
    print('%.4f %.4f %.4f %.4f' % (MSE,RMSE,MAE,R2))

def set_seed(seed,cuda):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.manual_seed(seed)

def dateinf(series, n_test):
    lt = len(series)
    print('Training start',series[0])
    print('Training end',series[lt-n_test-1])
    print('Testing start',series[lt-n_test])
    print('Testing end',series[lt-1])

set_seed(seed_num,use_cuda)


class LogCoshLoss(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, pred, target):
        loss = torch.log(torch.cosh(pred - target))
        return torch.mean(loss)

class CharbonnierLoss(nn.Module):
    def __init__(self, eps=1e-6):
        super().__init__()
        self.eps = eps
    def forward(self, pred, target):
        diff = pred - target
        loss = torch.sqrt(diff**2 + self.eps**2)
        return torch.mean(loss)

class QuantileLoss(nn.Module):
    def __init__(self, q=0.5):
        super().__init__()
        self.q = q
    def forward(self, pred, target):
        e = target - pred
        return torch.mean(torch.max(self.q * e, (self.q - 1) * e))



class HybridLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.logcosh = LogCoshLoss()
    def forward(self, pred, target):
        return 0.4 * self.logcosh(pred, target) + 0.6 * F.mse_loss(pred, target)

# 結合 RMSE + RIC 方向性損失
def make_composite_loss():
    def loss_fn(pred, target):
        mse = F.mse_loss(pred, target)
        ric = spearmanr(pred.detach().cpu().numpy(), target.detach().cpu().numpy())[0]
        penalty = torch.tensor(1 - ric**2)  # 強化方向誤差
        return mse + 0.5 * penalty
    return loss_fn



class TopKPairwiseHingeLoss(nn.Module):
    def __init__(self, k=10, margin=1.0):
        super().__init__()
        self.k = k
        self.margin = margin

    def forward(self, scores, labels):
        """
        scores: [N] tensor of predicted values
        labels: [N] tensor of true values (for sorting reference)
        """
        # 取得 top-K 與 bottom-K 的 index
        topk_idx = torch.topk(labels, self.k).indices
        bottomk_idx = torch.topk(-labels, self.k).indices

        # 預測分數差異
        score_top = scores[topk_idx]
        score_bottom = scores[bottomk_idx]

        # 計算 pairwise hinge loss
        # 損失 = margin - (top - bottom)，期望 top > bottom
        diff = score_top.unsqueeze(1) - score_bottom.unsqueeze(0)
        hinge = F.relu(self.margin - diff)

        return hinge.mean()




# loss_fnc = LogCoshLoss()
# loss_fnc = CharbonnierLoss()
# loss_fnc = QuantileLoss()

loss_fnc = HybridLoss()
# loss_fnc = make_composite_loss()

# loss_fnc = TopKPairwiseHingeLoss(k=10, margin=1.0)

In [ ]:
def log_shape(name, tensor):
    print(f"[Shape] {name}: {tuple(tensor.shape)}")


class Net(nn.Module):
    def __init__(self, in_dim, out_dim=n_steps, use_pca=False, pca_dim=None, pca_components=None):
        super().__init__()
        self.use_pca = use_pca

        if self.use_pca:
            assert pca_dim is not None and pca_components is not None, "需提供 pca_dim 和 pca_components"
            self.pca_layer = nn.Linear(in_dim, pca_dim, bias=False)
            self.pca_layer.weight.data = torch.tensor(pca_components, dtype=torch.float32)
            self.pca_layer.weight.requires_grad = False
            in_dim = pca_dim  # PCA 之後輸入維度會改變

        self.input_proj = nn.Linear(in_dim, hidden)  # 把輸入轉成 hidden 維度

        self.config = MambaConfig(d_model=hidden, n_layers=layer, n_steps=out_dim)
        self.mamba = Mamba(self.config)

    def forward(self, x):
        x = x.squeeze(1)                      # → (B, D)

        if self.use_pca:
            x = self.pca_layer(x)            # → (B, pca_dim)

        x = self.input_proj(x)               # → (B, hidden)
        x = x.unsqueeze(1)                   # → (B, 1, hidden)

        x = self.mamba(x)                    # → (B, n_steps)
        

        return x



def spearman_metric(y_true, y_pred):
    return spearmanr(y_true.cpu().numpy(), y_pred.cpu().numpy())[0]


def PredictWithData(trainX, trainy, testX, save_dir, window=window,use_pca=False, pca_dim=None, pca_components=None, val_ratio=0.2, patience=patience, loss_fnc=None):
    clf = Net(in_dim=trainX.shape[1], out_dim=n_steps, use_pca=use_pca, pca_dim=pca_dim, pca_components=pca_components)
    opt = torch.optim.Adam(clf.parameters(),lr=lr,weight_decay=wd)
    
    # 分割資料集
    val_size = int(len(trainX) * val_ratio)
    X_train_raw = trainX[:-val_size]
    y_train_raw = trainy[:-val_size]
    X_val_raw   = trainX[-val_size:]
    y_val_raw   = trainy[-val_size:]

    # Horizon 可整除長度
    valid_len = len(y_train_raw) - (len(y_train_raw) % n_steps)
    val_len   = len(y_val_raw)   - (len(y_val_raw) % n_steps)

    # 裁切資料
    X_train = X_train_raw[:valid_len]
    y_train = y_train_raw[:valid_len]
    X_val   = X_val_raw[:val_len]
    y_val   = y_val_raw[:val_len]

    # 建立 horizon 分組
    X_train = X_train.reshape(-1, n_steps, X_train.shape[1])
    y_train = y_train.reshape(-1, n_steps)
    X_val   = X_val.reshape(-1, n_steps, X_val.shape[1])
    y_val   = y_val.reshape(-1, n_steps)

    # 取每組 horizon 的最後一個時間步作為模型輸入
    xt = torch.from_numpy(X_train[:, -1, :]).float().unsqueeze(1)  # (B, 1, feature_dim)
    yt = torch.from_numpy(y_train).float()                         # (B, n_steps)
    xv = torch.from_numpy(X_val[:, -1, :]).float().unsqueeze(1)
    yv = torch.from_numpy(y_val).float()

    # 測試資料不需要 horizon 重組
    x_test = torch.from_numpy(testX).float().unsqueeze(1)
    
    if loss_fnc is None:
        loss_fnc = F.mse_loss
    
    best_r2 = -float('inf')
    best_rmse = float('inf')
    best_state_dict = None
    best_epoch = 0

    for e in range(epochs):
        clf.train()
        pred = clf(xt)
        loss = loss_fnc(pred, yt)

        opt.zero_grad()
        loss.backward()
        opt.step()

        clf.eval()
        with torch.no_grad():
            val_pred = clf(xv)
            val_loss = loss_fnc(val_pred, yv)

            val_pred_np = val_pred.cpu().numpy()  # shape: (B, n_steps)
            val_true_np = yv.cpu().numpy()

            r2_list = [r2_score(val_true_np[:, i], val_pred_np[:, i]) for i in range(val_pred_np.shape[1])]
            spearman_list = [spearmanr(val_true_np[:, i], val_pred_np[:, i])[0] for i in range(val_pred_np.shape[1])]

            avg_r2 = np.mean(r2_list)
            avg_spearman = np.mean(spearman_list)


        print(f'Epoch {e:03d} | Avg R²: {avg_r2:.4f} | Avg Spearman: {avg_spearman:.4f}')


        if avg_r2 > best_r2:

            best_r2 = avg_r2  
            best_epoch = e
            best_state_dict = clf.state_dict()
            best_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")      
        elif e - best_epoch >= patience:
            print(f"Early stopping triggered at epoch {e}")
            break
        
        # print(f'Epoch {e:03d} | Val RMSE: {RMSE:.4f}')

        # if RMSE < best_rmse:
        #     best_rmse = RMSE
        #     best_epoch = e
        #     best_state_dict = clf.state_dict()
        #     best_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")      
        # elif e - best_epoch >= patience:
        #     print(f"Early stopping triggered at epoch {e}")
        #     break

    best_model_path = f"{save_dir}model_{best_timestamp}.pth"
    torch.save(best_state_dict, best_model_path)
    print(f"Best model saved to: {best_model_path}")
    
    
    val_pred_np = val_pred.cpu().numpy() if use_cuda else val_pred.numpy()
    val_true_np = yv.cpu().numpy()

    mse_list = [mean_squared_error(val_true_np[:, i], val_pred_np[:, i]) for i in range(n_steps)]
    rmse_list = [mse ** 0.5 for mse in mse_list]
    mae_list = [mean_absolute_error(val_true_np[:, i], val_pred_np[:, i]) for i in range(n_steps)]
    r2_list = [r2_score(val_true_np[:, i], val_pred_np[:, i]) for i in range(n_steps)]
    spearman_list = [spearmanr(val_true_np[:, i], val_pred_np[:, i])[0] for i in range(n_steps)]

    avg_rmse = np.mean(rmse_list)
    avg_mae = np.mean(mae_list)
    avg_r2 = np.mean(r2_list)
    avg_spearman = np.mean(spearman_list)


    # 儲存評估指標
    # 儲存評估指標
    # log_path = f"{save_dir}/evaluation_log.txt"
    # with open(log_path, 'a') as f:
    #     f.write(f"Model: model_{best_timestamp}.pth\n")
    #     f.write(f"  MSE: {MSE:.4f} | RMSE: {RMSE:.4f} | MAE: {MAE:.4f} | R²: {R2:.4f}\n")
    #     f.write("-" * 60 + "\n")
    log_path = f"{save_dir}/evaluation_log.txt"
    with open(log_path, 'a') as f:
        f.write(f"Model: model_{best_timestamp}.pth\n")
        f.write(f"  Avg RMSE: {avg_rmse:.4f} | Avg MAE: {avg_mae:.4f} | Avg R²: {avg_r2:.4f} | Avg Spearman: {avg_spearman:.4f}\n")
        f.write("-" * 60 + "\n")

    
    clf.load_state_dict(best_state_dict)
    clf.eval()
    mat = clf(x_test)
    if use_cuda: mat = mat.cpu()
    yhat = mat.detach().cpu().numpy() 
    return yhat

In [8]:
from sklearn.preprocessing import PowerTransformer
data = pd.read_csv(str(ts_code)+"_value"+'.csv')
data['trade_date'] = pd.to_datetime(data['trade_date'], format='%Y/%m/%d')

# 加入星期幾（0=星期一, 6=星期日）
data['day_of_week'] = data['trade_date'].dt.dayofweek

# 加入月份（1～12）
data['month'] = data['trade_date'].dt.month



# 拆離 label 欄位
close = data.pop('close_TW').values
ratechg = data['close_TW_roc'].values *100


pt = PowerTransformer(method='yeo-johnson')
ratechg = pt.fit_transform(ratechg.reshape(-1, 1))


data.drop(columns=['close_TW_roc'], inplace=True)

# 擷取有效特徵欄位區段
dat = data.iloc[:, 4:].values

In [9]:
dat.shape

(2520, 61)

In [3]:
from sklearn.preprocessing import PowerTransformer
data = pd.read_csv(str(ts_code)+"_value"+'.csv')
data['trade_date'] = pd.to_datetime(data['trade_date'], format='%Y/%m/%d')

# 加入星期幾（0=星期一, 6=星期日）
data['day_of_week'] = data['trade_date'].dt.dayofweek

# 加入月份（1～12）
data['month'] = data['trade_date'].dt.month



# 拆離 label 欄位
close = data.pop('close_TW').values
ratechg = data['close_TW_roc'].values *100


pt = PowerTransformer(method='yeo-johnson')
ratechg = pt.fit_transform(ratechg.reshape(-1, 1))


data.drop(columns=['close_TW_roc'], inplace=True)

# 擷取有效特徵欄位區段
dat = data.iloc[:, 4:].values

num_samples = dat.shape[0]

# 產生 horizon indicator（重複每個 horizon 一次）
horizon_ids = np.tile(np.arange(n_steps), num_samples).reshape(num_samples, n_steps)

# 將其合併成新特徵
dat = np.repeat(dat[:, np.newaxis, :], n_steps, axis=1)  # shape: [samples, horizon, features]
dat = np.concatenate([dat, horizon_ids[..., np.newaxis]], axis=2)

In [7]:
dat.shape

(2520, 10, 62)

In [ ]:
use_pca = False
pca_dim = 25


if use_pca:
    pca = PCA(n_components=pca_dim)
    pca.fit(dat)

    pca_components = pca.components_[:pca_dim]
    dat_pca = dat @ pca_components.T
else:
    dat_pca = dat 
    
trainX, testX = dat_pca[:-n_test], dat_pca[-n_test:]
trainy = ratechg[:-n_test]

In [ ]:
trainX[0][0]

In [ ]:
trainX.shape

In [ ]:
trainy.shape

In [ ]:
val_ratio=0.2
val_size = int(len(trainX) * val_ratio)
X_train_raw = trainX[:-val_size]
y_train_raw = trainy[:-val_size]
X_val_raw   = trainX[-val_size:]
y_val_raw   = trainy[-val_size:]

# Horizon 可整除長度
valid_len = len(y_train_raw) - (len(y_train_raw) % n_steps)
val_len   = len(y_val_raw)   - (len(y_val_raw) % n_steps)

# 裁切資料
X_train = X_train_raw[:valid_len]
y_train = y_train_raw[:valid_len]
X_val   = X_val_raw[:val_len]
y_val   = y_val_raw[:val_len]

# 建立 horizon 分組
X_train = X_train.reshape(-1, n_steps, X_train.shape[1])
y_train = y_train.reshape(-1, n_steps)
X_val   = X_val.reshape(-1, n_steps, X_val.shape[1])
y_val   = y_val.reshape(-1, n_steps)

# 取每組 horizon 的最後一個時間步作為模型輸入
xt = torch.from_numpy(X_train[:, -1, :]).float().unsqueeze(1)  # (B, 1, feature_dim)
yt = torch.from_numpy(y_train).float()                         # (B, n_steps)
xv = torch.from_numpy(X_val[:, -1, :]).float().unsqueeze(1)
yv = torch.from_numpy(y_val).float()

In [4]:
X = []
y = []

for i in range(window + n_steps, len(data)):
    X.append(dat[i - window - n_steps : i - n_steps])  # [15, features]
    y.append(ratechg[i - n_steps : i])                 # [10,]

X = np.array(X)  # shape: [samples, window, features]
y = np.array(y)  # shape: [samples, n_steps]

val_ratio = 0.2
val_size = int(len(X) * val_ratio)

X_train = X[:-val_size]
y_train = y[:-val_size]
X_val   = X[-val_size:]
y_val   = y[-val_size:]

# 轉換成 tensor
xt = torch.from_numpy(X_train).float()     # [B, window, features]
yt = torch.from_numpy(y_train).float()     # [B, n_steps]
xv = torch.from_numpy(X_val).float()
yv = torch.from_numpy(y_val).float()

In [5]:
x_test = torch.from_numpy(testX[-window:]).float().unsqueeze(0)  # shape: [1, window, features]


NameError: name 'testX' is not defined

In [6]:
xt.shape

torch.Size([1996, 15, 10, 62])

In [ ]:
yt.shape

In [ ]:
x_test.shape

In [ ]:
from sklearn.preprocessing import PowerTransformer
data = pd.read_csv(str(ts_code)+"_value"+'.csv')
data['trade_date'] = pd.to_datetime(data['trade_date'], format='%Y/%m/%d')

# 加入星期幾（0=星期一, 6=星期日）
data['day_of_week'] = data['trade_date'].dt.dayofweek

# 加入月份（1～12）
data['month'] = data['trade_date'].dt.month



# 拆離 label 欄位
close = data.pop('close_TW').values
ratechg = data['close_TW_roc'].values *100


pt = PowerTransformer(method='yeo-johnson')
ratechg = pt.fit_transform(ratechg.reshape(-1, 1))


data.drop(columns=['close_TW_roc'], inplace=True)

# 擷取有效特徵欄位區段
dat = data.iloc[:, 4:].values

num_samples = dat.shape[0]

# 產生 horizon indicator（重複每個 horizon 一次）
horizon_ids = np.tile(np.arange(n_steps), num_samples).reshape(num_samples, n_steps)

# 將其合併成新特徵
dat = np.repeat(dat[:, np.newaxis, :], n_steps, axis=1)  # shape: [samples, horizon, features]
dat = np.concatenate([dat, horizon_ids[..., np.newaxis]], axis=2)

use_pca = False
pca_dim = 25


if use_pca:
    pca = PCA(n_components=pca_dim)
    pca.fit(dat)

    pca_components = pca.components_[:pca_dim]
    dat_pca = dat @ pca_components.T
else:
    dat_pca = dat 
    
trainX, testX = dat_pca[:-n_test], dat_pca[-n_test:]
trainy = ratechg[:-n_test]
# trainy = pd.Series(trainy).ewm(span=5).mean().values

In [ ]:


predictions = PredictWithData(trainX, trainy, testX, save_dir, loss_fnc=loss_fnc)
time = data['trade_date'][-n_test:]
data1 = close[-n_test:]
finalpredicted_stock_price = []
pred = close[-n_test-1]
for i in range(n_test):
    pred = close[-n_test-1+i]*(1+predictions[i])
    finalpredicted_stock_price.append(pred)
    
    
dateinf(data['trade_date'],n_test)
print('MSE RMSE MAE R2')
evaluation_metric(data1, finalpredicted_stock_price)
plt.figure(figsize=(10, 6))
plt.plot(time, data1, label='Stock Price')
plt.plot(time, finalpredicted_stock_price, label='Predicted Stock Price')
plt.title('Stock Price Prediction')
plt.xlabel('Time', fontsize=12, verticalalignment='top')
plt.ylabel('Close', fontsize=14, horizontalalignment='center')
plt.legend()
plt.show()

In [ ]:
x_test = torch.from_numpy(testX).float().unsqueeze(1)

clf = Net(in_dim=trainX.shape[1], out_dim=n_steps, use_pca=use_pca)
clf.load_state_dict(torch.load("0730_2330/model_20250730_092957.pth"))
clf.eval()
with torch.no_grad():
    predictions = clf(x_test)
testy = ratechg[-n_test:]


In [ ]:
testy

In [ ]:
predictions

In [ ]:
for i in range(predictions.shape[1]):
    plt.scatter(range(len(predictions)), predictions[:, i], label=f'horizon {i+1}')
plt.legend()
plt.title("Prediction over Time for Different Horizons")


In [ ]:
testy = ratechg[-n_test:]

def create_horizon_labels(ratechg_segment, horizon=n_steps):
    labels = []
    for i in range(len(ratechg_segment) - horizon + 1):
        labels.append(ratechg_segment[i:i+horizon])
    return np.array(labels)

testy = create_horizon_labels(ratechg[-n_test:], horizon=n_steps)
for i in range(testy.shape[1]):
    print(f"Horizon {i+1} 平均報酬率: {testy[:, i].mean():.4f}")


In [ ]:
pred_raw = predictions  # 暫存原始輸出
print("Shape before stack:", pred_raw.shape)

predictions = np.stack([
    pt.inverse_transform(pred_raw[:, i].reshape(-1, 1)).flatten()
    for i in range(pred_raw.shape[1])
], axis=1)

predictions = predictions / 100
testy = pt.inverse_transform(testy.reshape(-1, 1))
testy = testy.flatten() / 100 
print(predictions.shape)
print(testy.shape)

In [ ]:
def create_horizon_labels(ratechg_segment, horizon=n_steps):
    labels = []
    for i in range(len(ratechg_segment) - horizon + 1):
        labels.append(ratechg_segment[i:i+horizon])
    return np.array(labels)

# 重新生成 testy 對應未來三天的報酬率
testy = create_horizon_labels(ratechg[-n_test:], horizon=n_steps)
print(testy.shape)  # 應該是 (350, 3) 或 (348, 3)，看長度而定


In [ ]:
import matplotlib.pyplot as plt

y_pred = predictions
y_pred = y_pred[:341]

print("testy:", testy.shape)
print("y_pred:", y_pred.shape)


for i in range(n_steps):
    plt.figure(figsize=(6, 6))
    plt.scatter(testy[:, i], y_pred[:, i], alpha=0.6, color='steelblue', edgecolor='k')
    min_val = min(testy[:, i].min(), y_pred[:, i].min())
    max_val = max(testy[:, i].max(), y_pred[:, i].max())
    plt.plot([min_val, max_val], [min_val, max_val], 'r--', label='y = x')
    plt.xlabel(f"True Return (t+{i+1})")
    plt.ylabel(f"Predicted Return (t+{i+1})")
    plt.title(f"Horizon t+{i+1} Residual Scatter")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()


In [ ]:
import seaborn as sns
sns.histplot(y_pred.flatten(), bins=100, kde=True)


In [ ]:
residuals = y_pred.flatten() - testy.flatten()
sns.histplot(residuals, bins=100, kde=True)
